In [16]:
import pandas as pd
import boto3
import json

In [17]:
#Load DWH Params from a file
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('IAM_ROLE','KEY')
SECRET                 = config.get('IAM_ROLE','SECRET')

CLUSTER_TYPE       = config.get("DWH","CLUSTER_TYPE")
NUM_NODES          = config.get("DWH","NUM_NODES")
NODE_TYPE          = config.get("DWH","NODE_TYPE")

CLUSTER_IDENTIFIER = config.get("DWH","CLUSTER_IDENTIFIER")
DB_NAME          = config.get("DWH","DB_NAME")
DB_USER            = config.get("DWH","DB_USER")
DB_PASSWORD        = config.get("DWH","DB_PASSWORD")
PORT               = config.get("DWH","PORT")

IAM_ROLE_NAME      = config.get("DWH", "IAM_ROLE_NAME")

(DB_USER, DB_PASSWORD, DB_NAME)

pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER", "DB_NAME", "DB_USER", "DB_PASSWORD", "PORT", "IAM_ROLE_NAME"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE,CLUSTER_IDENTIFIER, DB_NAME, DB_USER, DB_PASSWORD, PORT, IAM_ROLE_NAME]
             })

,Param,Value
0,CLUSTER_TYPE,multi-node
1,NUM_NODES,4
2,NODE_TYPE,dc2.large
3,CLUSTER_IDENTIFIER,SparkifyCluster
4,DB_NAME,sparkify
5,DB_USER,sparkify_user
6,DB_PASSWORD,Passw0rd
7,PORT,5439
8,IAM_ROLE_NAME,sparkifydwhRole


In [18]:
#Create clients for IAM, S3 and Redshift
import boto3

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [20]:
#Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::715055115345:role/sparkifydwhRole


In [21]:
#Create a RedShift Cluster
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [22]:
#cluster status
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,sparkifycluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,sparkify_user
4,DBName,sparkify
5,Endpoint,"{'Address': 'sparkifycluster.c4uwhyzb5grx.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-00403878
7,NumberOfNodes,4


In [23]:
#cluster endpoint and role ARN
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  sparkifycluster.c4uwhyzb5grx.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::715055115345:role/sparkifydwhRole
